# Mobile Testbed


## 1.1 Background

Mobile, Alabama is a metropolitan city located in southwest Alabama (see figures below). The City of Mobile serves as the county seat for Mobile County; one of only two Alabama counties located on the Gulf Coast. According to the 2020 US Census, the population within the City of Mobile was 187,041 making it the fourth most populous city in Alabama, after Huntsville, Birmingham and Montgomery.  The City, Urban and Metropolitan areas are 180 mi2, 222.8 mi2 and 1,644 mi2, respectively.  The average elevation is +10 ft MSL.  The median household income is approximately 71% of the national average.  The Mobile metropolitan area consists of 430,197 residents. Baldwin County on the east side of Mobile Bay adds an additional 223,000 residents (est. 2019) to the metropolitan area, many of whom work in Mobile, bringing the total to approximately 653,000. As of 2011, approximately 1.2 million people lived within a 60-mile radius of Mobile. Major Utility Providers include Alabama Power, Mobile Area Water and Sewer System, Spire Natural Gas and various telecommunication companies - AT&T, Comcast, Verizon, etc.

Mobile’s waterfront location has long played an important role in its history. Situated along the Mobile River and adjacent to the Mobile Bay estuary, Mobile’s waterfront is accessible by deep draft ships through the Mobile Ship Channel, a Federally authorized navigation channel that connects Mobile River to the Gulf Intracoastal Waterway and the Gulf of Mexico. The deep water access supports robust waterfront commerce in the form of private and State port facilities, numerous ship manufacturers and service providers, industries supporting the offshore oil and gas industry, a cruise terminal, and many other activities

## 1.2 General Information

Mobile is a commercial and industrial hub on the Gulf Coast, supporting an intermodal network of sea, air, rail, tunnel, and surface transportation infrastructure.  Notable industrial facilities include the bulk and container terminals managed by the Alabama State Port Authority, the Alabama Cruise Terminal and the 1,650-acre Mobile Aeroplex at Brookley (previously Brookley Air Force Base), which supports a major Airbus final assembly line facility for the Airbus A220 and A320 commercial aircraft.  In addition to the port, shipbuilding began to make a major comeback in Mobile in 1999 with the founding of Austal USA.  These industrial facilities are served by four Class I railroads (Canadian National Railway (CNR), CSX Transportation (CSX), the Kansas City Southern Railway (KCS), and the Norfolk Southern Railway (NS), three  Interstates 10, 65 and 165, the confluence of three major state highways (90, 98, 45) and the George Wallace and Bankhead tunnels under the Mobile River, and. The George Wallace twin tunnels, which support daily traffic of over 70,000 vehicles.  This intermodal infrastructure has a tremendous economic impact on the City and State. The Port of Mobile is the 12th largest port in the US in terms of tonnage, and the Alabama State Port Authority adds an estimated economic value of over $25 billion/year and supports, directly and indirectly, over 154,000 jobs.  Mobile also serves the central Gulf Coast as a regional center for medicine.  

<img src="images/Mobile_County_Alabama_Incorporated_and_Unincorporated_areas_Mobile_Highlighted_01500004-01.png"> 

<center> City of Mobile, Alabama (red shading) and Incorporated Areas (gray shading) (Figure source: https://en.wikipedia.org/wiki/Mobile,_Alabama)


## 1.3 Purpose of Mobile, AL Testbed

Resilience assessment of coastal industrial city of moderate size susceptible to changing climate

Multiple hazards and vulnerabilities:
- Tropical cyclones – winds, storm surge
- Coastal flooding
- Precipitation extremes
- Sea Level Rise (SLR) 

Resilience modeling of intermodal transportation hub:
- Seaport
- Rail
- Roads
- Airport

## 1.4 More information about Mobile, AL Testbed

Hazard models and datasets used in this testbed come from:
1. Adhikari P, Abdelhafez MA, Dong Y, Guo Y, Mahmoud HN and Ellingwood BR (2021) Achieving Residential Coastal Communities Resilient to Tropical Cyclones and Climate Change. Front. Built Environ. 6:576403. https://doi.org/10.3389/fbuil.2020.576403
2. Abdelhafez M.A., Ellingwood B., Mahmoud H. (2021) Vulnerability of seaports to hurricanes and sea level rise in a changing climate: A case study for Mobile, AL. Coast. Eng. , Article 103884,
https://doi.org/10.1016/j.coastaleng.2021.103884
3. Abdelhafez, M.A., Ellingwood, B. & Mahmoud, H. (2022) Hidden costs to building foundations due to sea level rise in a changing climate. Sci Rep 12, 14020. https://doi.org/10.1038/s41598-022-18467-3



## 1.5 Prerequisites

The following modules are necessary to run this notebook. To ensure dependencies are correct, install all modules through **conda**.

| Module | Version | Notes |
| --- | --- | --- |
| pyIncore | =>1.3.0 | see: https://incore.ncsa.illinois.edu/doc/incore/install_pyincore.html |
| pyIncore_viz | =>1.5.0 | see: https://incore.ncsa.illinois.edu/doc/pyincore_viz/index.html |
| matplotlib | 3.1.2 | used for plotting results |

## 2. Building Damage Analysis
The following code is preparing the IN-CORE analysis by checking versions and connecting to IN-CORE web service.

In [ ]:
from pyincore import IncoreClient, Dataset, FragilityService, MappingSet, DataService
from pyincore.analyses.buildingstructuraldamage import BuildingStructuralDamage
from pyincore.analyses.montecarlolimitstateprobability import MonteCarloLimitStateProbability

import os
import pandas as pd
import numpy as np
import geopandas as gpd # For reading in shapefiles
import matplotlib.pyplot as plt
from IPython.display import display

import sys # For displaying package versions
import os # For managing directories and file paths if drive is mounted

from pyincore_viz.geoutil import GeoUtil as viz
from pyincore_viz.plotutil import PlotUtil as plot

client = IncoreClient()
client.clear_cache()

In [ ]:
# create data_service object for loading files
data_service = DataService(client)

In [ ]:
# Check package versions - good practice for replication
print("Python Version ",sys.version)
print("pandas version: ", pd.__version__)
print("numpy version: ", np.__version__)

### 2.1 Hurricane's storm surge modeling
Two major hurricanes have affected the Alabama (AL) coastal region in the vicinity of Mobile significantly in the past two decades: Ivan (2004) and Katrina (2005). However, the surge induced by Katrina (at 3.4 to 3.8 m in downtown Mobile) resulted in disabled roads and inundated areas and was larger than the surge from Ivan. Accordingly, the impact of a number of hurricane scenarios constructed from Hurricane Katrina will be considered in performing a damage assessment of buildings in Mobile County.

Katrina initiated as a tropical depression on 23 August 2005 near the Bahamas. Late on August 25, its strength increased to hurricane strength, and at 1200 August 28, Katrina was a Category 5 hurricane over the Gulf of Mexico, with maximum winds of 75 m/s and a central pressure of 909 millibars. Katrina’s strength waned at its second landfall on the following day to Category 3 at 1100 August 29 over southeast Louisiana and Mississippi, with a maximum wind speed of 57 m/s and a central pressure of 920 millibars.

Hydrologic and hydrodynamic models were developed to model the hurricane hazard. The hydrologic analysis was conducted on Delft3d. To simulate the hurricane wind proﬁle throughout its track, the analysis domain must include the point of hurricane formation to capture the water propagated from the Gulf of Mexico (GOM) accurately. Accordingly, a nesting grid modeling technique is used to simulate a broad selection of spatial scales. A large lower resolution grid of about 10 km is chosen to cover the GOM region as the ﬁrst stage of the analysis. Then, the output information from the GOM model is passed to a higher resolution grid of about 2 km that covers the northern part of the Gulf (NG). Finally, the results of the previous two domains are passed to the highest resolution grid domain of about 0.38 km surrounding Mobile Bay (MB). Bathymetry and topography data are based on the U.S. Coastal Relief Model (CRM) which has a horizontal resolution of three arc seconds (approximately 90 m). The vertical datum in the CRM model is mean sea level. In areas where the CRM data are not available, the General Bathymetric Chart of the Oceans (900 m horizontal resolution) is used. The seaﬂoor roughness factor is deﬁned by the spatial variations of the Manning coefﬁcient, which is extracted from the land use data using the National Land Cover Database (NLCD) and then transformed to compatible Manning’s values. (Adhikari et al., 2021)

<img src="images/Fig3.png"> 

<center> Modeling domains, hurricane Katrina’s original, shifted track, and their landfall locations (Figure source: Abdelhafez et al., 2021)

### 2.2 Validation of the hazard modeling
The accuracy of the storm surge model from Delft3D was tested using data obtained during Hurricane Katrina from the NOAA Tides and Currents stations at Dauphin Island, AL and at Pensacola, FL, as they were the only two stations that had recorded data during the hurricane. The simulation also captures the peak storm surge of about 3.5m in downtown Mobile, which matches the FEMA high water marks at this location More details about the validation can be found in Adhikari et al. (2021).

<img src="images/Fig4.png"> 

<center> (a) Dauphin island and (b) Pensacola, FL observation stations. (Figure source: Adhikari et al., 2021)

### 2.3 Storm surge coupled with the projected sea level rise
The actual hydrodynamic parameters of water level and characteristics of the existing hurricane track are used in the baseline storm surge scenario, illustrated in the previous Figure, while for the future scenarios, the water level and characteristics were updated by the Army Corps of Engineers for Dauphin Island gauge station based on NOAA climate projections scenarios. Two sea level rise scenarios are selected to represent the intermediate and extreme scenarios in NOAA’s scale. (Adhikari et al., 2021)

<img src="images/SLR_projections2-07-07-07.png"> 

<center> NOAA's global mean sea level rise projection (Figure source: Abdelhafez et al., 2021)



A raster map of the developed storm surge hazard coupled with projected SLR are shown in the figure below. The hazard data are available in terms of scenario-based with the following IDs in IN-CORE:

* Water level for natural hurricane Katrina in 2005 ID: 6420c6e6d9ae37665ff0d5f2
* Water level for natural hurricane Katrina coupled with NOAA intermediate SLR ID: 64222491d9ae37665ff0d5f3
* Water level for natural hurricane Katrina coupled with NOAA extreme SLR ID: 6422253cd9ae37665ff0d5f4


* Water level for shifted hurricane Katrina ID: 642d387bfde0f316c3493530
* Water level for shifted hurricane Katrina coupled with NOAA intermediate SLR ID: 642d38f1fde0f316c3493531
* Water level for shifted hurricane Katrina coupled with NOAA extreme SLR ID: 642d3c7763601e1e19241468

<img src="images/Figure_12_1.png"> 

<center> Inundations for (A) natural Hurricane Katrina track and (B) shifted Hurricane Katrina track. Blue color dominates the area inundated by the hurricane, red represents the extra area ﬂooded by the hurricane coupled with NOAA intermediate SLR, and the green shows the inundated area by the hurricane coupled with NOAA extreme SLR (Figure source: Adhikari et al., 2021)
    

The following code reads the storm surge hazard data 

In [ ]:
# Surge Inundation Hazard Map based on the Hurricane Katrina in 2005 from IN-CORE
hazard_type = "Hurricane Surge"
Hurricane_Inundation_level_m_id = "6420c6e534810d74880b511e"   # Raster Map for the water levels in m

#visualize the surge Inundation hazard data 
Hurricane_water_m_dataset = Dataset.from_data_service(Hurricane_Inundation_level_m_id, DataService(client))
map1=viz.map_raster_overlay_from_file(Hurricane_water_m_dataset.get_file_path('tif'))
map1

In [ ]:
# add opacity control - NOTE: It takes time before the opacity takes effect.
map1.layers[1].interact(opacity=(0.0,1.0,0.01))

In [ ]:
dataset = Dataset.from_data_service(Hurricane_Inundation_level_m_id, DataService(client))
map = viz.plot_raster_file_with_legend(dataset.get_file_path('tif'))
map

### 2.4 Exposure Modeling

### Residential buildings dataset
Data related to the building inventory in Mobile County, AL, were constructred by integrating the Microsoft Footprint (Bing Maps dataset), ATTOM (a real estate information database), and The National Structure Inventory (NSI). More information about the integration between the Microsoft Footprint and ATTOM datasets can be found in Abdelhafez et al., 2022. NSI was the main contributor to the residential buildings dataset, while ATTOM data were used when NSI data were missing. 

In [ ]:
# Mobile Residential Building Inventory from IN-CORE
Building_Inventory_id = "649361d8e435fd233fbbd1b3"

# visualize the building inventory
Mobile_Building_Inventory = Dataset.from_data_service(Building_Inventory_id, DataService(client))
map=viz.plot_map(Mobile_Building_Inventory, column="archetype", category=True, basemap=True)
map

In [ ]:
# load building inventory as Geodataframe
filename = Mobile_Building_Inventory.get_file_path('shp')
print("The IN-CORE Dataservice has saved the Building Inventory on your local machine: "+filename)
bldg_inv_gdf = gpd.read_file(filename)
bldg_inv_gdf.head()

In [ ]:
# Classification of the 165810 residential buildings in Mobile County 

bldg_inv_gdf.groupby(["arch_flood"], as_index=False)["guid"].count()

In [ ]:
# Galveston Hurricane flood fragility mappings for buildings
mapping_id = "62fefd688a30d30dac57bbd7"
fragility_service = FragilityService(client)
mapping_set = MappingSet(fragility_service.get_mapping(mapping_id))

### 2.5 Building Damage Analysis


In [ ]:
# Reading the Hazard type and the Hazard ID for the six hurricane and SLR scenarios
hazard_type = "hurricane" 

hurricane_hazard_dict = {}
hurricane_hazard_dict[0] = {'id': "6420c6e6d9ae37665ff0d5f2", 'name':'Natural Katrina'} 
hurricane_hazard_dict[1] = {'id': "64222491d9ae37665ff0d5f3", 'name':'Natural Katrina + INT SLR'}
hurricane_hazard_dict[2] = {'id': "6422253cd9ae37665ff0d5f4", 'name':'Natural Katrina + EXT SLR'}
hurricane_hazard_dict[3] = {'id': "642d387bfde0f316c3493530", 'name':'Shifted Katrina'}
hurricane_hazard_dict[4] = {'id': "642d38f1fde0f316c3493531", 'name':'Shifted Katrina + INT SLR'}
hurricane_hazard_dict[5] = {'id': "642d3c7763601e1e19241468", 'name':'Shifted Katrina + EXT SLR'}


In [ ]:
bldg_dmg = BuildingStructuralDamage(client)

bldg_dmg.load_remote_input_dataset("buildings", Building_Inventory_id)
bldg_dmg.set_input_dataset("dfr3_mapping_set", mapping_set)

In [ ]:
# Run the building damage analysis for the six hurricane and SLR scenarios

output_result=[]
output=[]
for i in range(0,len(hurricane_hazard_dict)): 
    result = []
    result2 = []
    result_name = "Mobile_bldg_natural_hurricane_dmg_result{0}".format(i)

    bldg_dmg.set_parameter("fragility_key", "Non-Retrofit Fragility ID Code")
    bldg_dmg.set_parameter("result_name", result_name)
    bldg_dmg.set_parameter("hazard_type", hazard_type)
    bldg_dmg.set_parameter("hazard_id", hurricane_hazard_dict[i]['id'])
    bldg_dmg.set_parameter("num_cpu", 8)
    bldg_dmg.run_analysis()
    # Retrieve result dataset
    result = bldg_dmg.get_output_dataset('ds_result')
    # Convert dataset to Pandas DataFrame
    result2 = result.get_dataframe_from_csv(low_memory=False)
    output.append(result2)
    output_result.append(result)
building_dmg_result_MC = output_result
building_dmg_result = output

In [ ]:
# Select only the buildings exposed to the hazard sceanrios
output3=[]
for j in range(0,len(building_dmg_result)):
    output2 = building_dmg_result[j][building_dmg_result[j]['haz_expose']=='yes']
    output3.append(output2)
bdmg_df_expose = output3
    
    
# Add 'DS_max' attribute to bdmg_df_expose that provide the max damage state for each Mobile residential building
for j in range(0,len(bdmg_df_expose)):
    bdmg_df_expose[j]['DS_max'] = bdmg_df_expose[j].loc[:,['DS_0', 'DS_1', 'DS_2', 'DS_3']].idxmax(axis = 1)

In [ ]:
# bdmg_df_expose[2][bdmg_df_expose[2]['DS_1']>0.01]

In [ ]:
# bdmg_df_expose[5][bdmg_df_expose[5]['guid']=='cf0baa3f-9331-479e-805d-24958dbaec0d']

In [ ]:
output5=[]
output7=[]
for k in range(0,len(bdmg_df_expose)):
    output4 = bdmg_df_expose[k]['DS_max'].value_counts(normalize=True).mul(100).index.tolist()
    output6 = bdmg_df_expose[k]['DS_max'].value_counts(normalize=True).mul(100).tolist()
    output5.append(output4)
    output7.append(output6)
    indexes = output5
    values  = output7
    

# Set plot parameters
fig, ax = plt.subplots(figsize=(20, 10), dpi=300)
width = 0.15 # width of bar
x = np.arange(len(indexes[0]))
x_labels = ['DS_0', 'DS_1', 'DS_2', 'DS_3']


containers = ax.bar(x , values[0], width, color='#000080', label='Natural Katrina')
containers = ax.bar(x  + width, values[1], width, color='#0F52BA', label='Natural Katrina + INT SLR')
containers = ax.bar(x  + (2 * width), values[2], width, color='#6593F5', label='Natural Katrina + EXT SLR')
containers = ax.bar(x  + (3 * width), values[3], width, color='#73C2FB', label='Shifted Katrina')
containers = ax.bar(x  + (4 * width), values[4], width, color='#81C6EB', label='Shifted Katrina + INT SLR')
containers = ax.bar(x  + (5 * width), values[5], width, color='#D4F1F7', label='Shifted Katrina + EXT SLR')



for bars in ax.containers:
    ax.bar_label(bars, labels = [f'{x.get_height()/100:.1%}' for x in bars], padding=5, fontsize = 15)
    
    
ax.set_ylabel('Percentage of the buildings (%)', labelpad=15, fontsize = 20)
ax.set_ylim(0,110)
ax.set_xticks(x + width + width/2)
ax.set_xticklabels(x_labels, fontsize = 20)
ax.set_xlabel('Damage State', labelpad=15, fontsize = 20)
ax.set_title('Distribution of most probable damage state for residential buildings',fontsize = 25)
ax.legend(fontsize = 20)
# ax('ytick', labelsize=10)    # fontsize of the tick labels
plt.tick_params(labelsize=20)
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

fig.tight_layout()
plt.show()

In [ ]:
# To visualize the output of Natural Katrina

bldg_results = pd.merge(bldg_inv_gdf, building_dmg_result[2], how = 'right', left_on = ['guid'], right_on=['guid'])

In [ ]:
# To show the probability of exceedance of DS_1 > 0

bldg_results_F1 = bldg_results[(bldg_results['archetype'] > 0) & (bldg_results['DS_1'] > 0) ]
bldg_results_F1.explore(column='DS_1',cmap='Reds', 
                        popup=['guid','g_elev','ffe_elev','LS_0','LS_1','LS_2','DS_1','DS_2'
                                ],
                        tooltip=['guid','g_elev','ffe_elev','LS_0','LS_1','LS_2','DS_1','DS_2'
                                ],
                        tiles='CartoDB positron',
                        style_kwds=dict(color="Red",weight=5, opacity=0.4))

## 3. Monte Carlo Simulation (MCS)


In [ ]:
output_mc=[]
num_samples = 500

for j in range(0,len(building_dmg_result_MC)):

    mc_bldg = MonteCarloLimitStateProbability(client)

    mc_bldg.set_input_dataset("damage", building_dmg_result_MC[j])  #  Load the Mobile building damage results dataset 
                                                             # generated from the previous model as an input
    mc_bldg.set_parameter("num_cpu", 8)
    mc_bldg.set_parameter("num_samples", num_samples)
    mc_bldg.set_parameter("damage_interval_keys", ["DS_0", "DS_1", "DS_2", "DS_3"])
    mc_bldg.set_parameter("failure_state_keys", ["DS_1", "DS_2", "DS_3"])

    mc_bldg.set_parameter("result_name", "storm_surge_mc_failure_probability_buildings") # name of csv file with results
    mc_bldg.run_analysis()  # Run the Monte Carlo Simulation module to obtain the building failure probabilities. 
    building_failure_probability = mc_bldg.get_output_dataset('failure_probability')  # get buildings failure probabilities
    result_mc = building_failure_probability.get_dataframe_from_csv()
    output_mc.append(result_mc)
df_bldg_fail = output_mc


In [ ]:
df_bldg_fail[0][df_bldg_fail[0]['failure_probability']>0]

In [ ]:
# To show the failure_probability Due to Katina scenario 
bldg_func_gdf = bldg_inv_gdf.merge(df_bldg_fail[0], on='guid')
viz.plot_gdf_map(bldg_func_gdf, column='failure_probability')


In [ ]:
df_bldg_fail_1 = df_bldg_fail[0][['guid','failure_probability']]
df_bldg_fail_2 = df_bldg_fail[1][['failure_probability']]
df_bldg_fail_3 = df_bldg_fail[2][['failure_probability']]
df_bldg_fail_4 = df_bldg_fail[3][['failure_probability']]
df_bldg_fail_5 = df_bldg_fail[4][['failure_probability']]
df_bldg_fail_6 = df_bldg_fail[5][['failure_probability']]

df_bldg_fail_1.rename(columns = {'failure_probability':'fp_1'}, inplace = True) #Natural Katrina
df_bldg_fail_2.rename(columns = {'failure_probability':'fp_2'}, inplace = True) #Natural Katrina + INT SLR
df_bldg_fail_3.rename(columns = {'failure_probability':'fp_3'}, inplace = True) #Natural Katrina + EXT SLR
df_bldg_fail_4.rename(columns = {'failure_probability':'fp_4'}, inplace = True) #Shifted Katrina
df_bldg_fail_5.rename(columns = {'failure_probability':'fp_5'}, inplace = True) #Shifted Katrina + INT SLR
df_bldg_fail_6.rename(columns = {'failure_probability':'fp_6'}, inplace = True) #Shifted Katrina + EXT SLR
bldg_inv_gdf_all = pd.concat((df_bldg_fail_1,df_bldg_fail_2,df_bldg_fail_3,df_bldg_fail_4,df_bldg_fail_5,df_bldg_fail_6), axis=1)


bldg_inv_gdf_final = bldg_inv_gdf.merge(bldg_inv_gdf_all, on='guid')



In [ ]:
bldg_inv_gdf_nan = bldg_inv_gdf_final
bldg_inv_gdf_nan.replace(0, np.nan, inplace=True)

In [ ]:
#To get number of Builidngs with failure_probability >0 for fp_1 to fp_6
bldg_inv_gdf_group = bldg_inv_gdf_nan.groupby(["archetype"], as_index=False).count()
bldg_inv_gdf_group = bldg_inv_gdf_group[['archetype','nsi_id','fp_1','fp_2','fp_3','fp_4','fp_5','fp_6']]
bldg_inv_gdf_group